In [2]:
!pip install jupyter_contrib_nbextensions
!jupyter contrib nbextension install - user
from jedi import settings
settings.case_insensitive_completion = True

Please supply at least one subcommand: install, migrate, uninstall


In [1]:
USER_FLAG = "--user"
# Install ai platform and kfp
!pip3 install {USER_FLAG} google-cloud-aiplatform==1.3.0 --upgrade
!pip3 install {USER_FLAG} kfp --upgrade
!pip install google_cloud_pipeline_components

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 61.0 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.3/304.3 kB 7.1 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 636.6/636.6 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 64.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.9/152.9 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 10.2 MB/s eta 0:00:00
  Preparing metadata (setu

In [2]:
import os

if not os.getenv("IS_TESTING"):
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

In [1]:
!gcloud services enable compute.googleapis.com         \
                       containerregistry.googleapis.com  \
                       aiplatform.googleapis.com  \
                       cloudbuild.googleapis.com \
                       cloudfunctions.googleapis.com \
                       cloudresourcemanager.googleapis.com


Operation "operations/acat.p2-953811166431-bd45a9dd-b3bb-4a28-a026-bd64f83a06d4" finished successfully.


In [10]:
from typing import NamedTuple
from kfp.v2 import dsl
from kfp.v2.dsl import (Artifact,
                        Dataset,
                        Input,
                        Model,
                        Output,
                        Metrics,
                        ClassificationMetrics,
                        component, 
                        OutputPath, 
                        InputPath)

from kfp.v2 import compiler
from google.cloud import bigquery
from google.cloud import aiplatform
from google.cloud.aiplatform import pipeline_jobs
from google_cloud_pipeline_components import aiplatform as gcc_aip
from datetime import datetime

In [3]:
USER_FLAG = "--user"
REGION="us-central1"

#Setting the environment path
PATH=%env PATH
%env PATH={PATH}:/home/jupyter/.local/bin
# to get the projet name
shell_output=!gcloud config get-value project 2> /dev/null
PROJECT_ID=shell_output[0]

# Set bucket name to store the artifacts
BUCKET_NAME="gs://"+PROJECT_ID+"-bucket-employee"

PIPELINE_ROOT = f"{BUCKET_NAME}/pipeline_root_employee/"
PIPELINE_ROOT



env: PATH=/opt/conda/bin:/opt/conda/condabin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games:/home/jupyter/.local/bin


'gs://vertex-ai-gcp-1-bucket-employee/pipeline_root_employee/'

In [4]:
!python -m pip install --user virtualenv
!echo "create env"
!python -m venv vertex_venv
#Adding kernel to jupyter
!echo "Add kernel to jupyter"
!ipython kernel install --name "vertex_env" --user

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 58.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.5/468.5 kB 43.9 MB/s eta 0:00:00
create env
Add kernel to jupyter
Installed kernelspec vertex_env in /home/jupyter/.local/share/jupyter/kernels/vertex_env


In [5]:
#First Component in the pipeline to fetch data from big query.
#Table1 data is fetched
@component(
    packages_to_install=["google-cloud-bigquery==2.34.1", "pandas", "pyarrow","db-dtypes","scikit-learn"],
    base_image="python:3.7",
    output_component_file="data_fetch_bigquery.yaml"
)
def get_data(
    bq_table : str,
    output_data_path: Output[Dataset]
):
    from google.cloud import bigquery
    import pandas as pd

    bqclient = bigquery.Client()
    table = bigquery.TableReference.from_string(
        bq_table
    )
    rows = bqclient.list_rows(
        table,
    )
    dataframe = rows.to_dataframe(
        create_bqstorage_client=True,
    )
    dataframe.to_csv(output_data_path.path + ".csv" , index=False, encoding='utf-8-sig')

In [6]:
#Second component in the pipeline to to combine data from 2 sources and for some data transformation
@component(
    packages_to_install=["scikit-learn", "pandas"],
    base_image="python:3.7",
    output_component_file="data_transformation.yaml",
)
def data_transformation(
    dataset: Input[Dataset],
    dataset_train: Output[Dataset],
    dataset_test: Output[Dataset]
):
    
    from sklearn.metrics import roc_curve
    from sklearn.model_selection import train_test_split
    from joblib import dump
    from sklearn.metrics import confusion_matrix
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.ensemble import RandomForestClassifier
    import pandas as pd
    
    data = pd.read_csv(dataset.path+".csv")
    data = data.copy()
    data = data.drop(['region','employee_id'],axis=1)
    data = data.dropna(axis=0)
    data['gender'] = data['gender'].replace({'f':0, 'm':1})
    dummies = pd.get_dummies(data['department'])
    data = pd.concat([data,dummies],axis=1)
    data = data.drop('department',axis=1)
    edu_ranking = {"Master's & above" : 1, "Bachelor's" : 2 , "Below Secondary" : 3}
    recruit_ranking = {'sourcing' : 1, 'other' : 2, 'referred' : 3}
    data['education'] = data['education'].map(edu_ranking)
    data['recruitment_channel'] = data['recruitment_channel'].map(recruit_ranking)
    train, test = train_test_split(data, test_size=0.3)
    train.to_csv(dataset_train.path + ".csv" , index=False, encoding='utf-8-sig')
    test.to_csv(dataset_test.path + ".csv" , index=False, encoding='utf-8-sig')

In [7]:
#Third component Model training
@component(
    packages_to_install = [
        "pandas",
        "scikit-learn",
    ], base_image="python:3.9",
)
def emp_promotion_training(
    dataset:  Input[Dataset],
    model: Output[Model], 
):
    
    from sklearn.ensemble import RandomForestClassifier
    import pandas as pd
    import pickle

    data = pd.read_csv(dataset.path+".csv")
    model_rf = RandomForestClassifier(n_estimators=10)
    model_rf.fit(
        data.drop(columns=["is_promoted"]),
        data.is_promoted,
    )
    model.metadata["framework"] = "Random Forest"
    file_name = model.path + f".pkl"
    with open(file_name, 'wb') as file:  
        pickle.dump(model_rf, file)

In [27]:
#Fourth component Model evaluation
@component(
    packages_to_install = [
        "pandas",
        "scikit-learn"
    ], base_image="python:3.9",
)
def model_evaluation(
    test_set:  Input[Dataset],
    trained_model: Input[Model],
    metrics: Output[ClassificationMetrics],
    kpi: Output[Metrics]
):

    from sklearn.ensemble import RandomForestClassifier
    import pandas as pd
    import logging 
    import pickle
    from sklearn.metrics import roc_curve, confusion_matrix, accuracy_score, recall_score, precision_score
    import json
    import typing


    data = pd.read_csv(test_set.path+".csv")
    model = RandomForestClassifier()
    file_name = trained_model.path + ".pkl"
    with open(file_name, 'rb') as file:  
        model = pickle.load(file)
    
    y_test = data.drop(columns=["is_promoted"])
    y_target=data.is_promoted
    y_pred = model.predict(y_test)
    

    y_scores =  model.predict_proba(data.drop(columns=["is_promoted"]))[:, 1]
    fpr, tpr, thresholds = roc_curve(
         y_true=data.is_promoted.to_numpy(), y_score=y_scores, pos_label=True
    )
    metrics.log_roc_curve(fpr.tolist(), tpr.tolist(), thresholds.tolist())  
    
    metrics.log_confusion_matrix(
       ["False", "True"],
       confusion_matrix(
           data.is_promoted, y_pred
       ).tolist(), 
    )
    
    accuracy = accuracy_score(data.is_promoted, y_pred.round())
    recall=recall_score(data.is_promoted, y_pred.round(), pos_label=1)
    precision=precision_score(data.is_promoted, y_pred.round(), pos_label=1)
    trained_model.metadata["accuracy"] = float(accuracy)
    trained_model.metadata["recall"] = float(recall)
    trained_model.metadata["precision"] = float(precision)
    kpi.log_metric("accuracy", float(accuracy))
    kpi.log_metric("recall", float(recall))
    kpi.log_metric("precision", float(precision))


In [41]:
@dsl.pipeline(
    # Default pipeline root. You can override it when submitting the pipeline.
    pipeline_root=PIPELINE_ROOT,
    # A name for the pipeline. Use to determine the pipeline Context.
    name="employee-pipeline",
    
)
def pipeline(
    bq_table: str = "vertex-ai-gcp-1.employee_data.employee_promotion_data",
    project: str = PROJECT_ID,
    region: str = REGION, 
    display_name: str = 'pipeline-employeepromotion-rf-job{}'.format(datetime.now().strftime("%Y%m%d%H%M%S")),
    ):
    
    data = get_data(bq_table)
    data_transformed=data_transformation(data.outputs["output_data_path"])
    train_model = emp_promotion_training(data_transformed.outputs["dataset_train"])
    
    model_evaluation_op = model_evaluation(
        test_set=data_transformed.outputs["dataset_test"],
        trained_model=train_model.outputs["model"],
    )

In [42]:
compiler.Compiler().compile(pipeline_func=pipeline,package_path='employee_promotion_prediction_rf.json')

In [43]:
pipeline_run_rf = pipeline_jobs.PipelineJob(
    display_name="employee_promotion_ml_pipeline_rf",
    template_path="employee_promotion_prediction_rf.json",
    enable_caching=False,
    location=REGION,
)

In [44]:
pipeline_run_rf.run()

Creating PipelineJob
PipelineJob created. Resource name: projects/953811166431/locations/us-central1/pipelineJobs/employee-pipeline-20221116115400
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/953811166431/locations/us-central1/pipelineJobs/employee-pipeline-20221116115400')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/employee-pipeline-20221116115400?project=953811166431
PipelineJob projects/953811166431/locations/us-central1/pipelineJobs/employee-pipeline-20221116115400 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/953811166431/locations/us-central1/pipelineJobs/employee-pipeline-20221116115400 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/953811166431/locations/us-central1/pipelineJobs/employee-pipeline-20221116115400 current state:
PipelineState.PIPELINE_STATE_CANCELLING
PipelineJob run completed. Resource name: projects/95381